# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
print(len(df))
df.head()

8188


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [10]:
df = pd.read_csv( 'homepage_actions.csv')

df.id.value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [12]:
df.head(25)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [13]:
df.describe(include=['object'])

,timestamp,group,action
count,8188,8188,8188
unique,8188,2,2
top,2017-01-17 16:26:01.904792,control,view
freq,1,4264,6328


In [16]:
df['click'] = 0
df['click'].loc[df['action'] == 'click'] = 1

/Users/charlielosche/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [37]:
#Your code here
click_f = df['action'] == 'click'
e_group_f = df['group'] == 'experiment'
c_group_f = df['group'] == 'control'

control_df = df.loc[c_group_f].copy()
experiment_df = df.loc[e_group_f].copy()
print('P is equal to: ', 0.05348777384439879/2)
stats.ttest_ind(control_df['click'], experiment_df['click'], equal_var=False)

P is equal to:  0.026743886922199395


Ttest_indResult(statistic=-1.9312441445072492, pvalue=0.05348777384439879)

In [23]:
# Fixed it to have 1 and 0 so there isa different mean - There is no meaningful different between control and experimewnt
# It's a two tailed test, so the one tail would be .025 - below .05 alpha - to check which tail that p applies to:

print(control_df['click'].mean())
print(experiment_df['click'].mean())

0.21857410881801126
0.23649337410805302


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [29]:
len(experiment_df.loc[experiment_df['click'] == 1]) * .21857410881801126

202.83677298311446

In [30]:
#Your code here
control_rate = control_df['click'].mean()
exp_experiment_rate = len(experiment_df['click']) * control_rate
act_experiment_rate = experiment_df['click'].mean()

print(control_rate)
print(exp_experiment_rate)
print(act_experiment_rate)

0.21857410881801126
857.6848030018762
0.23649337410805302


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [31]:
#Your code here
n = len(experiment_df['click'])
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

25.888551746650283


In [34]:
actual_experiment_clicks = experiment_df['click'].sum()
z_score = (actual_experiment_clicks - exp_experiment_rate)/std
print(z_score)

2.71607302278784


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [35]:
#Your code here
p_val = stats.norm.sf(z_score)

p_val

0.003303067275926571

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.